In [2]:
############################################
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt

# Read the data
with nc.Dataset('tb_sfc_precip_2017_1_4_7_10.nc') as f:
    tb=f.variables['tb'][:]
    sfc_precip=f.variables['sfc_precip'][:]
    a=np.nonzero(tb[:,0]==tb[:,0])

tb=tb[a[0],:]
tb_mean=np.mean(tb,axis=0)
tb_std=np.std(tb,axis=0)




#[169.75692749023438 93.41938781738281 194.39111328125 128.16268920898438,221.89120483398438 221.03565979003906 163.11097717285156,267.5454406738281 240.66546630859375]

In [5]:
X_scaled=(tb-tb_mean)/tb_std
X_train=X_scaled[::10,:]
print(X_train.shape)

(967427, 9)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Define the Autoencoder class
class SimpleAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(SimpleAutoencoder, self).__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(True),
            nn.Linear(hidden_dim, latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(True),
            nn.Linear(hidden_dim, input_dim)  # Use Sigmoid for normalized output
        )

    def forward(self, x):
        # Encode
        x = self.encoder(x)
        
        # Decode
        x = self.decoder(x)
        return x

# Assuming X_train is a PyTorch tensor with shape (n_sample, 9)
input_dim = X_train.shape[1]
hidden_dim = 16  # Hidden layer dimension
latent_dim = 3  # Latent dimension


In [9]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) 

# Initialize the autoencoder
autoencoder = SimpleAutoencoder(input_dim, hidden_dim, latent_dim)  

criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

In [10]:




# Define loss function and optimizer

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    avg_loss = 0
    for data in train_loader:
        inputs = data[0]
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        avg_loss += loss.item()
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss/len(train_loader):.4f}')

print("Training complete.")

Epoch [1/20], Loss: 0.0146
Epoch [2/20], Loss: 0.0042
Epoch [3/20], Loss: 0.0036
Epoch [4/20], Loss: 0.0033
Epoch [5/20], Loss: 0.0033
Epoch [6/20], Loss: 0.0032
Epoch [7/20], Loss: 0.0032
Epoch [8/20], Loss: 0.0032
Epoch [9/20], Loss: 0.0032
Epoch [10/20], Loss: 0.0032
Epoch [11/20], Loss: 0.0032
Epoch [12/20], Loss: 0.0031
Epoch [13/20], Loss: 0.0031
Epoch [14/20], Loss: 0.0031
Epoch [15/20], Loss: 0.0031
Epoch [16/20], Loss: 0.0031
Epoch [17/20], Loss: 0.0031
Epoch [18/20], Loss: 0.0031
Epoch [19/20], Loss: 0.0031
Epoch [20/20], Loss: 0.0031
Training complete.


In [11]:
scripted_model = torch.jit.script(autoencoder)
scripted_model.save("autoencoder.pt")
import pickle
d_scaler={'mean':tb_mean,'std':tb_std}
pickle.dump(d_scaler,open('tb_scaler.pkl','wb'))